## Import

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
import requests
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


### Health topic (bbc)

In [ ]:
base_url = "https://www.bbc.com"
start_url = "https://www.bbc.com/news/health"

# base_url에 각 기사 뒷부분 주소 연결하여 각 기사의 url return
def get_articles_urls(soup):
    return [base_url + li.a["href"] for li in soup.select("li.lx-stream__post-container")]

# get_page_articles 함수 정의
def get_page_articles(driver):
    driver_source = driver.page_source
    soup = BeautifulSoup(driver_source, 'html.parser')
    article_urls = get_articles_urls(soup)

    # 각 기사의 url을 타고 들어가서 article_titles, article_contents 배열에 기사 제목, 내용 저장
    article_titles = []
    article_contents = []
    for article_url in article_urls:
        article_title, article_content = extract_single_article_content(article_url)
        if article_title and article_content:
            article_titles.append(article_title)
            article_contents.append(article_content)
    return article_titles, article_contents, soup

# url에 저장된 기사 제목, 추출하는 함수
def extract_single_article_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    title = soup.select_one("h1")
    if title:
        title = title.get_text(strip=True)
    else:
        return None, None

    paragraphs = []
    p_elements = soup.select("div.ssrcss-7uxr49-RichTextContainer.e5tfeyi1 > p")

    if not p_elements: # 비어있는 경우를 확인하여 에러를 방지합니다.
        return None, None

    last_p_element = p_elements[-1]
    # div.ssrcss-7uxr49-RichTextContainer.e5tfeyi1 클래스 하위의 p 요소들을 가져오기
    for p in p_elements:
        # 마지막 문단의 i, a 요소는 날려버려야 하는 경우가 있다.
        if p == last_p_element:
            # p 요소 안에 있는 모든 a와 i 태그 삭제
            for tag in p.find_all(["i", "a"]):
                tag.decompose()
        # p 요소의 텍스트만 추출하여 paragraphs에 추가
        paragraphs.append(p.text.strip())

    content = "\n".join(paragraphs)

    return title, content

# 크롬 드라이버 설치, 연결
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 웹 페이지 열기
url = "https://www.bbc.com/news/health"
driver.get(url)

# 데이터 프레임 초기화
df_health = pd.DataFrame(columns=["Title", "Content"])

# title과 content를 가져오기 위해 get_page_articles 함수 호출
article_titles, article_contents, soup = get_page_articles(driver)

while True:
    # 데이터 프레임에 저장
    for title, content in zip(article_titles, article_contents):
        if title in df_health:
            continue
        else:
            df_health = df_health.append({"Title": title, "Content": content}, ignore_index=True)

    if len(df_health) > 200:
            break

    print('다음페이지 ' + str(len(df_health)))
    time.sleep(5)
    # 페이지를 변경한 후에도 다음 뉴스 목록을 가져올 수 있도록 변경된 페이지에 대한 soup 객체 생성
    driver_source = driver.page_source
    page_soup = BeautifulSoup(driver_source, 'html.parser')

    # 변경된 페이지에 대한 정보를 크롤링하고 순환하는 데 사용
    article_titles, article_contents, soup = get_page_articles(driver)


# 웹 드라이버 종료
driver.quit()

# 데이터 프레임 출력
df_health

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26060\1651645511.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_health = df_health.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26060\1651645511.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_health = df_health.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26060\1651645511.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_health = df_health.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26060\1651645511.py:72: FutureWarning: Th

다음페이지 20


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26060\1651645511.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_health = df_health.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26060\1651645511.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_health = df_health.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26060\1651645511.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_health = df_health.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26060\1651645511.py:72: FutureWarning: Th

다음페이지 36


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26060\1651645511.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_health = df_health.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26060\1651645511.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_health = df_health.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26060\1651645511.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_health = df_health.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26060\1651645511.py:72: FutureWarning: Th

...

다음페이지 179


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26060\1651645511.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_health = df_health.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26060\1651645511.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_health = df_health.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26060\1651645511.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_health = df_health.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26060\1651645511.py:72: FutureWarning: Th

다음페이지 199


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26060\1651645511.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_health = df_health.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26060\1651645511.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_health = df_health.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26060\1651645511.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_health = df_health.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26060\1651645511.py:72: FutureWarning: Th

,Title,Content
0,"Taurine may extend life and health, scientists...","Taurine - a nutrient found in meat, fish and s..."
1,Weekend hot weather alert escalated to amber,A weekend heat-health alert has been raised fr...
2,Humza Yousaf 'disturbed' by claims NHSGGC spie...,"First Minister Humza Yousaf has said he was ""d..."
3,UK heatwave: How does sunscreen work?,A heat-health alert has been issued for parts ...
4,Heatwaves: What do they do to the body and who...,A heat-health alert has been issued for parts ...
...,...,...
213,Jersey-UK reciprocal health agreement updated,A reciprocal health agreement (RHA) between Je...
214,'My mum donating her kidney was a gift of life...,An 18-year-old medallist at the World Transpla...
215,Savannah Marshall: Boxer admits school struggl...,Former world champion boxer Savannah Marshall ...
216,Is the past year's spike in strike action here...,"Over the past 12 months, a historic wave of in..."


In [ ]:
df_health.describe()

,Title,Content
count,218,218
unique,202,202
top,Dementia patient failed by safeguarding system...,An elderly dementia patient was failed by the ...
freq,2,2


In [ ]:
# 중복 기사 제거, Topic 컬럼 추가
df_health = df_health.drop_duplicates()
df_health['Topic'] = 'health'

In [ ]:
# health topic의 데이터프레임
df_health

,Title,Content,Topic
0,"Taurine may extend life and health, scientists...","Taurine - a nutrient found in meat, fish and s...",health
1,Weekend hot weather alert escalated to amber,A weekend heat-health alert has been raised fr...,health
2,Humza Yousaf 'disturbed' by claims NHSGGC spie...,"First Minister Humza Yousaf has said he was ""d...",health
3,UK heatwave: How does sunscreen work?,A heat-health alert has been issued for parts ...,health
4,Heatwaves: What do they do to the body and who...,A heat-health alert has been issued for parts ...,health
...,...,...,...
213,Jersey-UK reciprocal health agreement updated,A reciprocal health agreement (RHA) between Je...,health
214,'My mum donating her kidney was a gift of life...,An 18-year-old medallist at the World Transpla...,health
215,Savannah Marshall: Boxer admits school struggl...,Former world champion boxer Savannah Marshall ...,health
216,Is the past year's spike in strike action here...,"Over the past 12 months, a historic wave of in...",health


In [ ]:
import pandas as pd
from openpyxl import Workbook

# Excel 파일로 저장하기
with pd.ExcelWriter('health.xlsx', engine='openpyxl') as writer:
    df_health.to_excel(writer, index=False, sheet_name='Sheet1')

### Technology Topic

In [ ]:
base_url = "https://www.bbc.com"
start_url = "https://www.bbc.com/news/technology"

def get_articles_urls(soup):
    return [base_url + li.a["href"] for li in soup.select("li.lx-stream__post-container")]

# get_page_articles 함수 정의
def get_page_articles(driver):
    driver_source = driver.page_source
    soup = BeautifulSoup(driver_source, 'html.parser')
    article_urls = get_articles_urls(soup)

    article_titles = []
    article_contents = []
    for article_url in article_urls:
        article_title, article_content = extract_single_article_content(article_url)
        if article_title and article_content:
            article_titles.append(article_title)
            article_contents.append(article_content)
    return article_titles, article_contents, soup

def extract_single_article_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    title = soup.select_one("h1")
    if title:
        title = title.get_text(strip=True)
    else:
        return None, None

    paragraphs = []
    p_elements = soup.select("div.ssrcss-7uxr49-RichTextContainer.e5tfeyi1 > p")

    if not p_elements: # 비어있는 경우를 확인하여 에러를 방지합니다.
        return None, None

    last_p_element = p_elements[-1]
    # div.ssrcss-7uxr49-RichTextContainer.e5tfeyi1 클래스 하위의 p 요소들을 가져오기
    for p in p_elements:
        # 마지막 문단의 a, i에 태그가 나올 땐 날려버려야 하는 경우가 있다.
        if p == last_p_element:
            # p 요소 안에 있는 모든 a와 i 태그 삭제
            for tag in p.find_all(["i", "a"]):
                tag.decompose()
        # p 요소의 텍스트만 추출하여 paragraphs에 추가
        paragraphs.append(p.text.strip())

    content = "\n".join(paragraphs)

    return title, content

# 크롬 드라이버 설치, 연결
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 웹 페이지 열기
url = "https://www.bbc.com/news/technology"
driver.get(url)

# 데이터 프레임 초기화
df_tech = pd.DataFrame(columns=["Title", "Content"])

# title과 content를 가져오기 위해 get_page_articles 함수 호출
article_titles, article_contents, soup = get_page_articles(driver)

while True:
    # 데이터 프레임에 저장
    for title, content in zip(article_titles, article_contents):
        if title in df_tech:
            continue
        else:
            df_tech = df_tech.append({"Title": title, "Content": content}, ignore_index=True)

    if len(df_tech) > 200:
            break

    print('다음페이지 ' + str(len(df_tech)))
    time.sleep(5)
    # 페이지를 변경한 후에도 다음 뉴스 목록을 가져올 수 있도록 변경된 페이지에 대한 soup 객체 생성
    driver_source = driver.page_source
    page_soup = BeautifulSoup(driver_source, 'html.parser')

    # 변경된 페이지에 대한 정보를 크롤링하고 순환하는 데 사용
    article_titles, article_contents, soup = get_page_articles(driver)


# 웹 드라이버 종료
driver.quit()

# 데이터 프레임 출력
df_tech

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\1139894182.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tech = df_tech.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\1139894182.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tech = df_tech.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\1139894182.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tech = df_tech.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\1139894182.py:72: FutureWarning: The frame.appe

다음페이지 20


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\1139894182.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tech = df_tech.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\1139894182.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tech = df_tech.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\1139894182.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tech = df_tech.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\1139894182.py:72: FutureWarning: The frame.appe

다음페이지 40


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\1139894182.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tech = df_tech.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\1139894182.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tech = df_tech.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\1139894182.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tech = df_tech.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\1139894182.py:72: FutureWarning: The frame.appe

...

다음페이지 170


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\1139894182.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tech = df_tech.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\1139894182.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tech = df_tech.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\1139894182.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tech = df_tech.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\1139894182.py:72: FutureWarning: The frame.appe

다음페이지 184


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\1139894182.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tech = df_tech.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\1139894182.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tech = df_tech.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\1139894182.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tech = df_tech.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\1139894182.py:72: FutureWarning: The frame.appe

,Title,Content
0,Reddit blackout: Subreddits to go private on M...,Thousands of Reddit communities will be inacce...
1,Cryptoqueen: Adviser to missing fugitive disap...,An FBI-wanted former spy and trusted adviser t...
2,University of Manchester hit by cyber attack,The University of Manchester has been targeted...
3,Facebook owner Meta plans to create Twitter rival,Meta has shown staff plans for a text-based so...
4,Northampton: Warehouse uses robots to help sta...,A business said it was using robots in its war...
...,...,...
199,ChatGPT: Can students pass using AI tools at u...,"As exam season gets under way, students might ..."
200,Vietnam to crack down on anonymous social medi...,Social media users in Vietnam must soon verify...
201,Disabled Downpatrick woman feels trapped in he...,A woman with a mobility impairment has said sh...
202,Tech layoffs: LinkedIn cuts 700 jobs and close...,LinkedIn has become the latest tech firm to ax...


In [ ]:
df_tech.describe()

,Title,Content
count,204,204
unique,204,204
top,Reddit blackout: Subreddits to go private on M...,Thousands of Reddit communities will be inacce...
freq,1,1


In [ ]:
df_tech['Topic'] = 'technology'

In [ ]:
import pandas as pd
from openpyxl import Workbook

# Excel 파일로 저장하기
with pd.ExcelWriter('technology.xlsx', engine='openpyxl') as writer:
    df_tech.to_excel(writer, index=False, sheet_name='Sheet1')

### Economy topic - 한 번에 저장 다 못해서 두번 나누고 concate

In [ ]:
base_url = "https://www.bbc.com"

def get_articles_urls(soup):
    return [base_url + li.a["href"] for li in soup.select("li.lx-stream__post-container")]

# get_page_articles 함수 정의
def get_page_articles(driver):
    driver_source = driver.page_source
    soup = BeautifulSoup(driver_source, 'html.parser')
    article_urls = get_articles_urls(soup)

    article_titles = []
    article_contents = []
    for article_url in article_urls:
        article_title, article_content = extract_single_article_content(article_url)
        if article_title and article_content:
            article_titles.append(article_title)
            article_contents.append(article_content)
    return article_titles, article_contents, soup

def extract_single_article_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    title = soup.select_one("h1")
    if title:
        title = title.get_text(strip=True)
    else:
        return None, None

    paragraphs = []
    p_elements = soup.select("div.ssrcss-7uxr49-RichTextContainer.e5tfeyi1 > p")

    if not p_elements: # 비어있는 경우를 확인하여 에러를 방지합니다.
        return None, None

    last_p_element = p_elements[-1]
    # div.ssrcss-7uxr49-RichTextContainer.e5tfeyi1 클래스 하위의 p 요소들을 가져오기
    for p in p_elements:
        if p == last_p_element:
            # p 요소 안에 있는 모든 a와 i 태그 삭제
            for tag in p.find_all(["i", "a"]):
                tag.decompose()
        # p 요소의 텍스트만 추출하여 paragraphs에 추가
        paragraphs.append(p.text.strip())

    content = "\n".join(paragraphs)

    return title, content

# 크롬 드라이버 설치, 연결
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 웹 페이지 열기
url = "https://www.bbc.com/news/business/economy"
driver.get(url)

# 데이터 프레임 초기화
df_economy = pd.DataFrame(columns=["Title", "Content"])

# title과 content를 가져오기 위해 get_page_articles 함수 호출
article_titles, article_contents, soup = get_page_articles(driver)

while True:
    # 데이터 프레임에 저장
    for title, content in zip(article_titles, article_contents):
        if title in df_economy:
            continue
        else:
            df_economy = df_economy.append({"Title": title, "Content": content}, ignore_index=True)

    if len(df_economy) > 200:
            break

    print('다음페이지 ' + str(len(df_economy)))
    time.sleep(5)
    # 페이지를 변경한 후에도 다음 뉴스 목록을 가져올 수 있도록 변경된 페이지에 대한 soup 객체 생성
    driver_source = driver.page_source
    page_soup = BeautifulSoup(driver_source, 'html.parser')

    # 변경된 페이지에 대한 정보를 크롤링하고 순환하는 데 사용
    article_titles, article_contents, soup = get_page_articles(driver)


# 웹 드라이버 종료
driver.quit()

# 데이터 프레임 출력
df_economy

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\4015638386.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy = df_economy.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\4015638386.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy = df_economy.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\4015638386.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy = df_economy.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\4015638386.py:71: FutureWarni

다음페이지 18


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\4015638386.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy = df_economy.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\4015638386.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy = df_economy.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\4015638386.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy = df_economy.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\4015638386.py:71: FutureWarni

다음페이지 38


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\4015638386.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy = df_economy.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\4015638386.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy = df_economy.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\4015638386.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy = df_economy.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\4015638386.py:71: FutureWarni

...

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\4015638386.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy = df_economy.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\4015638386.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy = df_economy.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\4015638386.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy = df_economy.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\4015638386.py:71: FutureWarni

다음페이지 76


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\4015638386.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy = df_economy.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\4015638386.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy = df_economy.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\4015638386.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy = df_economy.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\4015638386.py:71: FutureWarni

다음페이지 96


ConnectionError: HTTPSConnectionPool(host='www.bbc.comhttps', port=443): Max retries exceeded with url: //www.bbc.co.uk/usingthebbc/terms/can-i-share-things-from-the-bbc (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000208A2DD4CA0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
# 100개 정도밖에 저장이 안됐다.
df_economy

,Title,Content
count,96,96
unique,96,96
top,Brexit: New scheme will allow businesses to re...,A long-awaited post-Brexit scheme allowing Nor...
freq,1,1


In [ ]:
# 이어서 100개 저장
base_url = "https://www.bbc.com"

def get_articles_urls(soup):
    return [li.a["href"] if li.a["href"].startswith('http') else base_url + li.a["href"] for li in soup.select("li.lx-stream__post-container")]

# get_page_articles 함수 정의
def get_page_articles(driver):
    driver_source = driver.page_source
    soup = BeautifulSoup(driver_source, 'html.parser')
    article_urls = get_articles_urls(soup)

    article_titles = []
    article_contents = []
    for article_url in article_urls:
        article_title, article_content = extract_single_article_content(article_url)
        if article_title and article_content:
            article_titles.append(article_title)
            article_contents.append(article_content)
    return article_titles, article_contents, soup

def extract_single_article_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    title = soup.select_one("h1")
    if title:
        title = title.get_text(strip=True)
    else:
        return None, None

    paragraphs = []
    p_elements = soup.select("div.ssrcss-7uxr49-RichTextContainer.e5tfeyi1 > p")

    if not p_elements: # 비어있는 경우를 확인하여 에러를 방지합니다.
        return None, None

    last_p_element = p_elements[-1]
    # div.ssrcss-7uxr49-RichTextContainer.e5tfeyi1 클래스 하위의 p 요소들을 가져오기
    for p in p_elements:
        if p == last_p_element:
            # p 요소 안에 있는 모든 a와 i 태그 삭제
            for tag in p.find_all(["i", "a"]):
                tag.decompose()
        # p 요소의 텍스트만 추출하여 paragraphs에 추가
        paragraphs.append(p.text.strip())

    content = "\n".join(paragraphs)

    return title, content

# 크롬 드라이버 설치, 연결
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 웹 페이지 열기
url = "https://www.bbc.com/news/business/economy"
driver.get(url)

# 데이터 프레임 초기화
df_economy2 = pd.DataFrame(columns=["Title", "Content"])

# title과 content를 가져오기 위해 get_page_articles 함수 호출
article_titles, article_contents, soup = get_page_articles(driver)

while True:
    # 데이터 프레임에 저장
    for title, content in zip(article_titles, article_contents):
        if title in df_economy2:
            continue
        else:
            df_economy2 = df_economy2.append({"Title": title, "Content": content}, ignore_index=True)

    if len(df_economy2) > 120:
            break

    print('다음페이지 ' + str(len(df_economy2)))
    time.sleep(10)
    # 페이지를 변경한 후에도 다음 뉴스 목록을 가져올 수 있도록 변경된 페이지에 대한 soup 객체 생성
    driver_source = driver.page_source
    page_soup = BeautifulSoup(driver_source, 'html.parser')

    # 변경된 페이지에 대한 정보를 크롤링하고 순환하는 데 사용
    article_titles, article_contents, soup = get_page_articles(driver)


# 웹 드라이버 종료
driver.quit()

# 데이터 프레임 출력
df_economy2

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\2955085389.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy2 = df_economy2.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\2955085389.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy2 = df_economy2.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\2955085389.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy2 = df_economy2.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\2955085389.py:71: Futur

다음페이지 18


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\2955085389.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy2 = df_economy2.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\2955085389.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy2 = df_economy2.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\2955085389.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy2 = df_economy2.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\2955085389.py:71: Futur

다음페이지 36


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\2955085389.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy2 = df_economy2.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\2955085389.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy2 = df_economy2.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\2955085389.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy2 = df_economy2.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\2955085389.py:71: Futur

...

다음페이지 90


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\2955085389.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy2 = df_economy2.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\2955085389.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy2 = df_economy2.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\2955085389.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy2 = df_economy2.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\2955085389.py:71: Futur

다음페이지 110


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\2955085389.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy2 = df_economy2.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\2955085389.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy2 = df_economy2.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\2955085389.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_economy2 = df_economy2.append({"Title": title, "Content": content}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30476\2955085389.py:71: Futur

,Title,Content
0,Brexit: New scheme will allow businesses to re...,A long-awaited post-Brexit scheme allowing Nor...
1,Tesco could be breaking law on Clubcard pricin...,Tesco could be breaking the law over how it di...
2,Eurozone in recession as rising prices hit spe...,"The eurozone fell into recession this winter, ..."
3,"Rising rates to hit housing demand, builder says",The housing market could be heading for a furt...
4,Crypto: 24-hour cooling-off period included in...,Crypto companies must offer customers a 24-hou...
...,...,...
122,Stormont: Departments set to face large cuts t...,Stormont departments are facing large cuts in ...
123,Humza Yousaf: What will be in the new first mi...,The new SNP leader Humza Yousaf is set to be f...
124,Binance accused of breaking US financial laws,"US regulators are seeking to ban Binance, the ..."
125,John Lewis: Ex-boss says changing ownership mo...,A former John Lewis boss has said it would be ...


In [ ]:
# 데이터프레임을 합침
df_economy_combined = pd.concat([df_economy, df_economy2], ignore_index=True)

In [ ]:
df_economy_combined.describe()

,Title,Content
count,223,223
unique,205,205
top,Brexit: New scheme will allow businesses to re...,A long-awaited post-Brexit scheme allowing Nor...
freq,2,2


In [ ]:
# 중복 저장된 기사 제거
df_economy_combined = df_economy_combined.drop_duplicates()

In [ ]:
df_economy_combined.describe()

,Title,Content
count,205,205
unique,205,205
top,Brexit: New scheme will allow businesses to re...,A long-awaited post-Brexit scheme allowing Nor...
freq,1,1


In [ ]:
# topic 컬럼 추가
df_economy_combined['Topic'] = 'economy'

NameError: name 'df_economy_combined' is not defined

In [ ]:
# economy 주제 데이터프레임
df_economy_combined

,Title,Content,Topic
0,Brexit: New scheme will allow businesses to re...,A long-awaited post-Brexit scheme allowing Nor...,economy
1,Tesco could be breaking law on Clubcard pricin...,Tesco could be breaking the law over how it di...,economy
2,Eurozone in recession as rising prices hit spe...,"The eurozone fell into recession this winter, ...",economy
3,"Rising rates to hit housing demand, builder says",The housing market could be heading for a furt...,economy
4,Crypto: 24-hour cooling-off period included in...,Crypto companies must offer customers a 24-hou...,economy
...,...,...,...
218,Stormont: Departments set to face large cuts t...,Stormont departments are facing large cuts in ...,economy
219,Humza Yousaf: What will be in the new first mi...,The new SNP leader Humza Yousaf is set to be f...,economy
220,Binance accused of breaking US financial laws,"US regulators are seeking to ban Binance, the ...",economy
221,John Lewis: Ex-boss says changing ownership mo...,A former John Lewis boss has said it would be ...,economy


In [ ]:
import pandas as pd
from openpyxl import Workbook

# Excel 파일로 저장하기
with pd.ExcelWriter('economy.xlsx', engine='openpyxl') as writer:
    df_economy_combined.to_excel(writer, index=False, sheet_name='Sheet1')

### Social Impact Topic (로이터)

In [ ]:
# Reuters 사이트에 접속하기 위한 기본 URL 및 시작 URL
base_url = "https://www.reuters.com"
start_url = "https://www.reuters.com/tags/social-impact"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

# 해당 페이지의 기사 URL을 가져오는 함수
def get_articles_urls(soup):
    return [base_url + a['href'] for a in soup.find_all("a", {"data-testid": "Heading"})]

# 해당 페이지의 기사들의 제목과 내용을 가져오는 함수
def get_page_articles(driver):
    driver_source = driver.page_source
    soup = BeautifulSoup(driver_source, 'html.parser')
    article_urls = get_articles_urls(soup)

    article_titles = []
    article_contents = []
    for article_url in article_urls:
        article_title, article_content = extract_single_article_content(article_url)
        if article_title and article_content:
            article_titles.append(article_title)
            article_contents.append(article_content)
    return article_titles, article_contents, soup

# 단일 기사의 제목과 내용을 추출하는 함수
def extract_single_article_content(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    title_element = soup.find("h1")
    if title_element:
        title = title_element.text.strip()
    else:
        title = "기사 제목을 찾을 수 없습니다."

    content_paragraphs = [p for p in soup.find_all("p") if p.get("data-testid", "").startswith("paragraph-")]

    # 첫 번째 단락의 (Reuters) - 부분을 제거한 상태로 다시 저장
    if content_paragraphs and '(Reuters) - ' in content_paragraphs[0].text:
        content_paragraphs[0] = content_paragraphs[0].text.replace('(Reuters) - ', '')

    content = " ".join([p.text if not isinstance(p, str) else p for p in content_paragraphs])

    return title, content

# ChromeDriver 설치 및 웹 브라우저 실행
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 웹 페이지 열기
url = start_url
driver.get(url)

# 데이터 프레임 초기화
df_social = pd.DataFrame(columns=["Title", "Content"])

# title과 content를 가져오기 위해 get_page_articles 함수 호출
article_titles, article_contents, soup = get_page_articles(driver)

while True:
    # 데이터 프레임에 저장
    for title, content in zip(article_titles, article_contents):
        if title in df_social:
            continue
        else:
            df_social = df_social.append({"Title": title, "Content": content}, ignore_index=True)

    if len(df_social) > 200:
            break

    print('다음페이지 ' + str(len(df_social)))
    time.sleep(5)
    # 페이지를 변경한 후에도 다음 뉴스 목록을 가져올 수 있도록 변경된 페이지에 대한 soup 객체 생성
    driver_source = driver.page_source
    page_soup = BeautifulSoup(driver_source, 'html.parser')

    # 변경된 페이지에 대한 정보를 크롤링하고 순환하는 데 사용
    article_titles, article_contents, soup = get_page_articles(driver)


# 웹 드라이버 종료
driver.quit()

# 데이터 프레임 출력
df_social

In [ ]:
# 중복 기사 제거
df_social = df_social.drop_duplicates()

In [ ]:
# Topic 컬럼 추가
df_social['Topic'] = 'social_impact'

In [ ]:
# social_impact topic의 데이터프레임
df_social

In [ ]:
import pandas as pd
from openpyxl import Workbook

# Excel 파일로 저장하기
with pd.ExcelWriter('social_impact.xlsx', engine='openpyxl') as writer:
    df_social.to_excel(writer, index=False, sheet_name='Sheet1')